## Gaussian Puff Model

Yunha Lee 
August 7, 2024


# Pre-Step: Get weather data for the target area and times

In [ ]:
import pandas as pd
import xarray as xr

# Define the path and variables
weather_file_names = ['u_wind', 'v_wind', 'wind_direction', 'vertical_velocity', 'temperature']
data_dir = "/Users/yunhalee/Documents/methanDart/UCAR_reanalysis/"
geopotential_file = f"{data_dir}geopotential_height_NorthAmerica.nc"

# Define source locations and time range
source_locs = pd.DataFrame({
    'lat': [34.05, 34.06],
    'lon': [-118.25, -118.26],
    'height': [10, 50]  # Heights in meters
})
start_time = pd.to_datetime("2015-06-01 12:00:00")
end_time = pd.to_datetime("2015-06-10 12:00:00")

# Read geopotential height to find the corresponding pressure levels
with xr.open_dataset(geopotential_file) as ds:
    ds_gph = ds.sel(time=slice(start_time, end_time))

    # Dictionary to hold DataFrames for each location
    location_weather_data = {}

    # Iterate over each location
    for index, row in source_locs.iterrows():
        loc_id = f"lat{row['lat']}_lon{row['lon']}_height{row['height']}"
        loc_weather_data = pd.DataFrame()

        # Iterate over each weather variable
        for var_name in weather_file_names:
            with xr.open_dataset(f"{data_dir}{var_name}_NorthAmerica.nc") as ds_var:
                ds_var = ds_var.sel(time=slice(start_time, end_time))

                # Get the nearest geopotential heights for this location and convert heights to pressure levels
                gph_values = ds_gph.sel(latitude=row['lat'], longitude=row['lon'], method='nearest')

                # Print dimensions to debug
                print("Dimensions of gph_values:", gph_values.dims)

                # If multiple times are present, consider reducing them
                if 'time' in gph_values.dims:
                    gph_values = gph_values.isel(time=0)  # selecting the first time point for simplicity

                # Continue with height comparison
                height_diff = abs(gph_values['geopotential_height'] - row['height'])
                nearest_pressure_level_idx = height_diff.argmin(dim='isobaricInhPa')
                nearest_pressure_level = gph_values['isobaricInhPa'].isel(isobaricInhPa=nearest_pressure_level_idx).values

                # Print the type and value to debug
                print("Type of nearest_pressure_level:", type(nearest_pressure_level))
                print("Value of nearest_pressure_level:", nearest_pressure_level)


                var_data = ds_var.sel(latitude=row['lat'], longitude=row['lon'], isobaricInhPa=nearest_pressure_level, method='nearest')
                df_var = var_data.to_dataframe()

                # Renaming columns to reflect data specifics
                new_column_names = {
                    name: f"{name}"
                    for name in df_var.columns
                }

                df_var.rename(columns=new_column_names, inplace=True)
                if loc_weather_data.empty:
                    loc_weather_data = df_var
                else:
                    loc_weather_data = pd.merge(loc_weather_data, df_var, how='outer', left_index=True, right_index=True, suffixes=('', '_duplicate'))

                    # Optionally, drop duplicate columns if necessary
                    duplicate_columns = [col for col in loc_weather_data.columns if '_duplicate' in col]
                    loc_weather_data.drop(columns=duplicate_columns, inplace=True)

        # Save the DataFrame for this location
        location_weather_data[loc_id] = loc_weather_data

        # Save to CSV
        loc_weather_data.to_csv(f"../input_data/{loc_id}_weather_data.csv")

# Print one of the DataFrames as an example
for loc_id, df in location_weather_data.items():
    print(f"Data for {loc_id}:")
    print(df.head())


## Run the GPM model

In [ ]:
import numpy as np
from datetime import timedelta
from multiprocessing import Pool
import pandas as pd
from gaussian_puff_functions import *

def simulate_puff_concentration(source_index, source_locs, data, dt, emission_rate, times, chunk_size, area_size, height_levels, n_chunks):
    print(f"Source: {source_index + 1}/{len(source_locs)}")

    # Source location and conversion to UTM
    source_lat = source_locs.iloc[source_index]['lat']
    source_lon = source_locs.iloc[source_index]['lon']
    source_x, source_y = latlon_to_utm(source_lat, source_lon)
    source_z = source_locs.iloc[source_index]['height']

    # Define a 3D grid or area over which to calculate concentration
    grid_x, grid_y, grid_z = np.meshgrid(np.linspace(source_x-area_size/2, source_x+area_size/2, num=30),
                                         np.linspace(source_y-area_size/2, source_y+area_size/2, num=30),
                                         np.linspace(0, source_z + height_levels, num=10))  

    # Wind data and interpolation
    WS_x = data['u_wind']
    WS_y = data['v_wind']
    WA = data['wind_direction']
    WA_x = np.cos(WA)
    WA_y = np.sin(WA)
    WS = np.sqrt(WS_x**2 + WS_y**2)

    # Emission rates
    Q_truth = np.full((len(WS)), emission_rate)

    # Setup chunk processing
    n_ints = len(WS)-1 # ignore the last time 

    args = [(h, chunk_size, dt, n_ints, source_x, source_y, source_z, WS_x, WS_y, WS, Q_truth, grid_x, grid_y, grid_z, times) 
            for h in range(n_chunks)]
    
    print("args", len(args))

    print(f"chunk_size: {chunk_size}, n_chunks: {n_chunks}, dt: {dt}, n_ints: {n_ints}, "
      f"source_x: {source_x}, source_y: {source_y}, source_z: {source_z}, "
      f"WS_x length: {len(WS_x)}, WS length: {len(WS)}, Q_truth: {Q_truth[0]}, ")

    with Pool(processes=n_chunks) as pool:
        results = pool.map(process_chunk, args)
        #print("result", results)

    big_C = np.vstack(results)
    return big_C



##TODO GET_STABILITY_CLASS MUST TAKE SURFACE_WIND, not the U_Wind!! 

# Define source locations and time range
source_locs = pd.DataFrame({
    'lat': [34.05],
    'lon': [-118.25],
    'height': [10]  # Heights in meters
})

# Define the size of the area over which you want to simulate concentrations
area_size = 100  # domain size in meters (grid size is 100 meter)
height_levels = 100 # domain heigh in meters
dt = 1  # sec
emission_rate = 0.1  # kg/s

# Determine the simulation duration using chunk_size and n_chunk (for multiprocessing)
chunk_size = 10  # Number of minutes in each chunk
n_chunks = 6 # Number of chunks (also, number of processors)

start_time = pd.to_datetime("2015-06-01 12:00:00")
end_time = calculate_end_time(start_time, chunk_size, n_chunks)
print(f"The end time of the simulation will be: {end_time}")

# Create a sequence of minutes between start_time and end_time
times = pd.date_range(start=start_time, end=end_time, freq=f'{dt}s')

# Remove the last timestamp to exclude the last second (to make it clean number)
times = times[:-1]

print(times)

output_dir = "/Users/yunhalee/Documents/methanDart/Gaussian_Puff_CH4/output_data/"

# Prepare to store the results
all_data = []

# Iterate over each location
for s, row in source_locs.iterrows():
    loc_id = f"lat{row['lat']}_lon{row['lon']}_height{row['height']}"
    print(f"Data for {loc_id}:")

    data = pd.read_csv (f"../input_data/{loc_id}_weather_data.csv")

    # Convert the 'time' column to datetime if not already
    data['time'] = pd.to_datetime(data['time'])
    data.set_index('time', inplace=True)

    # Resample the data to hourly from 3-hourly data
    hourly_data = data.resample('1H').mean()
    hourly_data = hourly_data.interpolate(method='linear')

    # Filter the data between start_time and end_time
    filtered_data = hourly_data[start_time:end_time]

    resampled_data = filtered_data.resample(f'{dt}s').mean()
    resampled_data = resampled_data.interpolate(method='linear')

    # Print and/or process the interpolated data
    print("Interpolated weather data:", resampled_data.head())
    print("Interpolated weather data:", len(resampled_data))

    big_C = simulate_puff_concentration(s, source_locs,  resampled_data, dt, emission_rate, times, chunk_size, area_size, height_levels, n_chunks)

    plot_2d_concentration(big_C, times) 

    # Average the output
    output_dt_minutes = 60  # Define how many seconds you want to average over
    df_resampled_big_C = average_time_resolution(big_C, times, output_dt_minutes, output_dir, s, source_locs, area_size,height_levels)

    plot_2d_concentration_from_df(df_resampled_big_C)